### This is a classification problem... i think it is about bank marketing analysis (it wasn't specified but there is a similar dataset available on kaggle by the name)

# Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
df = pd.read_csv("../input/beginner-datasets/beginner_datasets/bank.csv")

# EDA

In [ ]:
df.head()

In [ ]:
df.poutcome.value_counts()

In [ ]:
df.info()

In [ ]:
df.describe()

### Lets go through each feature

### Ok guys, i want to explore some more plots today so i'll be referecing the seaborn documentation about how to plot data when one of the features is categorical...

https://seaborn.pydata.org/tutorial/categorical.html

In [ ]:
# catplot

sns.catplot(x="deposit", y="age", data=df);

# using jitter = False option doesn't give any info here and

As the size of the dataset grows, categorical scatter plots become limited in the information they can provide about the distribution of values within each category. When this happens, there are several approaches for summarizing the distributional information in ways that facilitate easy comparisons across the category levels

In [ ]:
#boxplot

sns.catplot(y="deposit", x="age", kind="box", data=df);

https://www.statisticshowto.com/probability-and-statistics/descriptive-statistics/box-plot/

The first is the familiar boxplot(). This kind of plot shows the three quartile values of the distribution along with extreme values. The “whiskers” extend to points that lie within 1.5 IQRs of the lower and upper quartile, and then observations that fall outside this range are displayed independently. This means that each value in the boxplot corresponds to an actual observation in the data.

**How to read? or what info you get?**

The box and whiskers chart shows you how your data is spread out. Five pieces of information (the “five number summary“) are generally included in the chart:

- The minimum (the smallest number in the data set). The minimum is shown at the far left of the chart, at the end of the left “whisker.” or sleeping T shape

- First quartile, Q1, is the far left of the box (or the far right of the left whisker).

- The median is shown as a line in the center of the box.

- Third quartile, Q3, shown at the far right of the box (at the far left of the right whisker).

- The maximum (the largest number in the data set), shown at the far right of the box

**Note on Outliers:**

Data sets can sometimes contain outliers that are suspected to be anomalies (perhaps because of data collection errors or just plain old flukes). If outliers are present, the whisker on the appropriate side is drawn to 1.5 * IQR rather than the data minimum or the data maximum. Small circles or unfilled dots are drawn on the chart to indicate where suspected outliers lie. Filled circles are used for known outliers.

other info... do read

https://www.purplemath.com/modules/boxwhisk3.htm#:~:text=The%20%22interquartile%20range%22%2C%20abbreviated,box%2Dand%2Dwhisker%20plot.&text=The%20IQR%20tells%20how%20spread,far%22%20from%20the%20central%20value.

In [ ]:
# hue = "education"

sns.catplot(x="deposit", y="age",hue="education", kind="box", data=df);

A related function, boxenplot(), draws a plot that is similar to a box plot but optimized for showing more information about the shape of the distribution. It is best suited for larger datasets:

In [ ]:
# kind="boxen"

sns.catplot(x="deposit", y="age",hue="education", kind="boxen", data=df);

In [ ]:
sns.catplot(x="education", y="age", hue="deposit",
            kind="violin", data=df);

https://mode.com/blog/violin-plot-examples/

The box plot is an old standby for visualizing basic distributions. It's convenient for comparing summary statistics (such as range and quartiles), but it doesn't let you see variations in the data. For multimodal distributions (those with multiple peaks) this can be particularly limiting.

But fret not—this is where the violin plot comes in. A violin plot is a hybrid of a box plot and a kernel density plot, which shows peaks in the data.

the same summary statistics as box plots:

 - the white dot represents the median

 - the thick gray bar in the center represents the interquartile range

 - the thin gray line represents the rest of the distribution, except for points that are determined to be “outliers” using a method that is a function of the interquartile range.

In [ ]:
# to be more space effecient use split=True

sns.catplot(x="education", y="age", hue="deposit",
            kind="violin",split=True, data=df);

In [ ]:
sns.catplot(x="education", y="age", hue="deposit", kind="bar", data=df);

In [ ]:
sns.catplot(x="education", kind="count", palette="ch:.25", data=df);

In [ ]:
sns.catplot(x="education", y="age", hue="deposit", kind="point", data=df);

A point plot represents an estimate of central tendency for a numeric variable by the position of scatter plot points and provides some indication of the uncertainty around that estimate using error bars.

In [ ]:
sns.violinplot(x=df.deposit, y=df.age)

In [ ]:
# using row instead of col cause there are a lot of values

sns.catplot(x="education", y="age", hue="deposit",
            row="job", aspect=0.7,
            kind="box", data=df);

In [ ]:
# got my hands on this cool code from the website abt violine plot

f, ax = plt.subplots(figsize=(8, 8))

# Show each distribution with both violins and points
sns.violinplot(x="deposit",y="age",data=df, palette="Set3", inner="points", bw =.2, cut=2, linewidth=3)

sns.despine(left=True)

f.suptitle('Deposit by age', fontsize=18, fontweight='bold')
ax.set_xlabel("Weight (g)",size = 16,alpha=0.7)
ax.set_ylabel("Feed",size = 16,alpha=0.7);

In [ ]:
# i have seen a guy decrease the coding by writing a function where he feels necessary so i am going to try the same...
# (i haven't been using functions a lot... it means i practice more but if i save time i can work on something else)
# a function to do the ploting:


def eda(col):
    print("--------------",col,"-----------------")
    if df[col].dtype == 'object':
        # bar
        df[col].value_counts().plot.bar(figsize=(10,5)); 
        plt.show()
        # count
        cat = sns.catplot(y=col, hue="deposit", kind="count",
            palette="pastel", edgecolor=".6",
            data=df);
        cat.set_xlabels(fontsize = 15)
        cat.set_ylabels(fontsize = 15)
        plt.show()
        print("ratio of yes/no based on category of " + col, end='\n\n')
        for cat in df[col].unique():
            print(cat,": " ,df.groupby([col]).deposit.value_counts()[cat]['yes']/df.groupby([col]).deposit.value_counts()[cat]['no'])
    else:
        sns.displot(df[col], color='r', kde=True,);
        plt.show()        
        sns.barplot(x=df['deposit'], y=df[col]);
        plt.show()        
        # box
        print("box")
        sns.catplot(y="deposit", x=col, kind="box", data=df);
        plt.show()
        # voilin
        sns.catplot(x="deposit", y=col,
            kind="violin",split=True, data=df);
        plt.show()
        sns.boxplot(x=df[col]);
        plt.show()
    print("\n\n\n\n")

In [ ]:
eda("age")

# right skewed
# there are obviously a lot of outliers...
# violine plot says the ones with has have a bit of greater density of outliers

In [ ]:
eda('job')
# gonna try one hot encoding...

In [ ]:
eda('marital')

In [ ]:
eda("education")

In [ ]:
eda("default")

In [ ]:
eda("balance")

In [ ]:
eda("housing")

In [ ]:
eda("loan")

In [ ]:
eda("contact")

In [ ]:
eda("day")
# we have 3 peaks

In [ ]:
eda("month")

In [ ]:
eda("duration")

In [ ]:
eda("campaign")

In [ ]:
eda("pdays")

In [ ]:
eda("previous")

In [ ]:
eda("poutcome")

In [ ]:
 cat = sns.catplot(y='deposit', kind="count",
            palette="pastel", edgecolor=".6",
            data=df);

In [ ]:
df['deposit'].value_counts().plot.bar(figsize=(10,5)); 

# Feature Engineering

### we can try to transform the features but logistic is not a good option here at all... most of the numeric data is really skewed, decision tree will work, if we use something like k nearest neighbours then it will be helpful if we perform feature scaling so lets just start off with that

In [ ]:
# get dataframe of categorical values after one hot encoding
encoder = OneHotEncoder(drop='first')
arry = []
for col in df.columns:
    if df[col].dtype == 'object':
        arry.append(pd.DataFrame(encoder.fit_transform(df[[col]]).toarray()).add_suffix(col))
        
cat_df = pd.concat([arry[0],arry[1],arry[2],arry[3],arry[4],arry[5],arry[6],arry[7],arry[8],arry[9]], axis=1)
cat_df.head()

In [ ]:
# get all numeric columns after normalizing
scaler = StandardScaler()
arry =[]
for col in df.columns:
    if df[col].dtype != 'object':
        arry.append(pd.DataFrame(scaler.fit_transform(df[[col]]), columns=[col]))
        
numeric_df = pd.concat([arry[0],arry[1],arry[2],arry[3],arry[4],arry[5],arry[6]], axis=1)
numeric_df

In [ ]:
y = cat_df['0deposit']
X = pd.concat([cat_df.drop(['0deposit'], axis=1), numeric_df], axis=1)

# Model building

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X ,y, test_size = 0.3, random_state=42)

In [ ]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train,y_train)
y_pred = dt_model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
dt_model.score(X_test,y_test)

# then random forest will perform better

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
rf_model.score(X_test,y_test)

In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train,y_train)
y_pred = knn_model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
knn_model.score(X_test,y_test)

In [ ]:
xg_model = XGBClassifier(verbosity = 0)
xg_model.fit(X_train,y_train)
y_pred = knn_model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
knn_model.score(X_test,y_test)

### There is a clustering approach that we can use... first we cluster the data then we determine which model performs best for each of the clusters.... later this way we can while predicting find the cluster a set of data belongs to and use the model which performs best for that cluster

and ofcourse we need more metrics then just accuracy score

In [ ]:
def clustering_approach(X,y, models,type = "none"):
    
    dfs = {}
    X_cls = {}
    y_cls = {}
    X_scaled = {}
    X_train, X_test, y_train, y_test = {},{},{},{}
    y_pred = {}
    models_out = {}
    
    # create knn model and predict
    knn_clf = KNeighborsClassifier()
    knn_clf.fit(X,y)
    df = pd.concat([X,y],axis=1) # so we can later separate x and y for each cluster
    df['knn_clf'] = knn_clf.predict(X)
    no_cls = knn_clf.classes_
    # get the dataframes, apply std.scaler, form train, test sets, apply models
    for cls in knn_clf.classes_:
        print("--------------The {} cluster's results-------------------".format(cls),end="\n\n")
        dfs[cls] = df[df['knn_clf'] == cls].iloc[:,:-1]
        
        X_cls[cls] = dfs[cls].iloc[:,:-1]
        y_cls[cls] = dfs[cls].iloc[:,-1]
        scaler = StandardScaler()
        X_scaled[cls] = scaler.fit_transform(X_cls[cls])
#         X_scaled[cls] = pd.DataFrame(X_scaled[cls],columns=df.columns[:-1])
    
        X_train[cls],X_test[cls],y_train[cls],y_test[cls] = train_test_split(X_scaled[cls],y_cls[cls],test_size=0.3,random_state=3)
        print("here")
        # type can be used for analyzing... eg: confusion matrix
        for model in models:
            model.fit(X_train[cls], y_train[cls])
            y_pred[cls] = model.predict(X_test[cls])
            print(model)
            print(model.score(X_test[cls],y_test[cls]))
            print(confusion_matrix(y_pred[cls], y_test[cls]), end="\n\n")
            models_out[str(model) + str(cls)] = model
            
    
    return [X_train, X_test, y_train, y_test,knn_clf, models_out]

In [ ]:
models = [LogisticRegression(), RandomForestClassifier(), KNeighborsClassifier()
          , XGBClassifier(verbosity = 0),LGBMClassifier(),SVC()] 


In [ ]:
X_train_clus, X_test_clus, y_train_clus, y_test_clus,clusterer, models = clustering_approach(X,y,models)

#### XGBoost performs the best individually on each cluster

# Thank you

### Don't forget to leave a like or upvote if this was worth your time